<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/paddy_rice_disease_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download(
    "tntiphan/paddy-rice-disease-classification"
)

print(dataset_path)

100%|██████████| 4.05G/4.05G [00:40<00:00, 107MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/tntiphan/paddy-rice-disease-classification/versions/7


In [7]:
import tensorflow as tf
import os

train_dir = os.path.join(dataset_path, "train")

# class names
class_names = sorted(os.listdir(train_dir))
class_count = len(class_names)
print("Classes:", class_names)

image_paths = []
labels = []

for idx, class_name in enumerate(class_names):
    class_folder = os.path.join(train_dir, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(idx)

image_paths = tf.constant(image_paths)
labels = tf.constant(labels)

Classes: ['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast']


In [8]:
import random

image_paths = []
labels = []

for class_name in class_names:
    class_folder = os.path.join(data_dir, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label2id[class_name])  # use JSON mapping

# Shuffle data
combined = list(zip(image_paths, labels))
random.shuffle(combined)
image_paths, labels = zip(*combined)
image_paths, labels = list(image_paths), list(labels)


In [9]:
total = len(image_paths)
train_split = int(total * 0.7)
val_split = int(total * 0.85)

train_paths, train_labels = image_paths[:train_split], labels[:train_split]
val_paths, val_labels = image_paths[train_split:val_split], labels[train_split:val_split]
test_paths, test_labels = image_paths[val_split:], labels[val_split:]

print("Train:", len(train_paths), "Val:", len(val_paths), "Test:", len(test_paths))


Train: 10703 Val: 2294 Test: 2294


In [10]:
import tensorflow as tf

IMG_SIZE = 224
BATCH_SIZE = 32

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img, label

def make_dataset(paths, labels):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.shuffle(1000)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_paths, train_labels)
val_ds = make_dataset(val_paths, val_labels)
test_ds = make_dataset(test_paths, test_labels)


In [11]:
import tensorflow as tf

IMG_SIZE = 224  # image size (matches our pipeline)
BATCH_SIZE = 32


In [12]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(class_count, activation='softmax')  # class_count = number of disease types
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
EPOCHS = 10  # you can increase later

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 116s 319ms/step - accuracy: 0.3315 - loss: 1.5826 - val_accuracy: 0.4621 - val_loss: 1.2362
Epoch 2/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 99s 297ms/step - accuracy: 0.5237 - loss: 1.1125 - val_accuracy: 0.6190 - val_loss: 0.9216
Epoch 3/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 94s 279ms/step - accuracy: 0.6849 - loss: 0.7834 - val_accuracy: 0.6870 - val_loss: 0.8122
Epoch 4/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 143s 282ms/step - accuracy: 0.7940 - loss: 0.5384 - val_accuracy: 0.7345 - val_loss: 0.7658
Epoch 5/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 93s 277ms/step - accuracy: 0.8572 - loss: 0.3691 - val_accuracy: 0.7772 - val_loss: 0.7159
Epoch 6/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 147s 292ms/step - accuracy: 0.9093 - loss: 0.2506 - val_accuracy: 0.7681 - val_loss: 0.8600
Epoch 7/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 92s 276ms/step - accuracy: 0.9418 - loss: 0.1654 - val_accuracy: 0.7406 - val_loss: 1.2123
Epoch 8/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 98s 292ms/step - accuracy: 0.9426 - loss

In [14]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)


72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 255ms/step - accuracy: 0.7756 - loss: 1.1813
Test Accuracy: 0.7750653624534607


In [16]:
model.save("rice_disease_cnn.keras")